In [16]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join

In [17]:
# load single example
def load_example( img_path ):

    Y = img_path[img_path.rfind('_')+1:-4]

    img = Image.open( img_path )
    img_mat = np.asarray( img )
    
    n_letters = len( Y )
    im_height = int(img_mat.shape[0])
    im_width = int(img_mat.shape[1]/n_letters)
    n_pixels = im_height*im_width
    
    X = np.zeros( [int(n_pixels+n_pixels*(n_pixels-1)/2),n_letters])
    for i in range(n_letters):
        
        # single letter
        letter = img_mat[:,i*im_width:(i+1)*im_width]/255
        
        # compute features
        x = letter.flatten()
        X[0:len(x),i] = x
        cnt = n_pixels 
        for j in range(0,n_pixels-1):
            for k in range(j+1,n_pixels):
                X[cnt,i] = x[j]*x[k]
                cnt = cnt + 1
           
        X[:,i] = X[:,i]/np.linalg.norm(X[:,i])
        
    return X, Y, img

# load all examples from a folder    
def load_examples( image_folder ):
    
    files = [f for f in listdir(image_folder) if isfile(join(image_folder, f))]

    X = []
    Y = []
    img = []
    for file in listdir(image_folder):
        path = join(image_folder, file)
        if isfile( path ):
                        
            X_,Y_,img_ = load_example( path )
            X.append( X_ )
            Y.append( Y_ )
            img.append( img_ )
        
    return X, Y, img

In [18]:
# Global Variables

training_size = 1000
testing_size = 500

alpha = 0.01

epochs = 30
features = 8256
labels = 26
letterDict = {}
L = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
for i in range(len(L)):
    letterDict[L[i]] = i

In [19]:
# Training Phase
def train(trn_X, trn_Y):
    
    # Add one column to each element of trn_X (for biases)
    X = np.vstack([np.hstack(trn_X), np.ones(np.hstack(trn_X).shape[1])])
    
    # Convert letters into numbers
    Y = np.hstack([np.array([letterDict.get(l) for l in word]) for word in trn_Y])
    
    # Parameters
    WB = np.zeros((labels, features+1)) # weight and biases
    
    for e in range(epochs):
        print(f"Epoch {e+1}", end="\r")
        flag = True
        
        for i in range (X.shape[1]):
            # Extract the feature vector
            iFeatures = X[:, i]
            # Matrixt multiplication
            mul = np.matmul(WB, iFeatures)
            # Get the predicted label
            p = np.argmax(mul)
            
            if p != Y[i]:
                flag = False
                
                # Update parameters
                temp1 = np.zeros((labels, X.shape[0]))
                temp2 = np.zeros((labels, X.shape[0]))
                temp1[Y[i], :] = 1
                temp2[p, :] = 1
                WB += (temp1 - temp2)*iFeatures
                
        if flag:
            # No more misclassifications
            print("\nOptimum reached")
            break
        
    return WB

# Testing Phase
def test(tst_X, WB):
    
    predictions = []
    for fm in tst_X:
        # Add an element to each character feature vector (for biases)
        fm = np.vstack([fm, np.ones(fm.shape[1])])
        
        # Matrix multiplication
        mul = np.matmul(WB, fm)
        
        # Get the predicted label
        p = np.argmax(mul, axis=0)
        predictions.append(p)
        
    return predictions
        

In [20]:
# load data

# load training examples
trn_X, trn_Y, trn_img = load_examples( 'ocr_names_images/trn' )

# load testing examples
tst_X, tst_Y, tst_img = load_examples( 'ocr_names_images/tst' )

In [21]:
print("Training")
print("...")
WB = train(trn_X, trn_Y)
print()
print("Weights and biases Learned!")
print()
print("Testing")
res = test(tst_X, WB)
print("Predictions done")


Training
...
Epoch 30
Weights and biases Learned!

Testing
Predictions done


In [22]:

# Convert letters into numbers
Y = [np.array([letterDict.get(char) for char in word]) for word in tst_Y]

# Sequence Error
num = 0
for i in range(testing_size):
    if(tuple(res[i]) != tuple(Y[i])):
        num+=1
print("Sequence error: ", f"{num/500:.4f}")

# Charachter Error
num = 0
word = 0
for y in Y:
    y = list(y)
    for char in range(len(y)):
        if(y[char] != list(res[word])[char]):
            num+=1
    word+=1
print("Charachter error: ", f"{num/sum([len(l) for l in tst_Y]):.4f}")

Sequence error:  0.7120
Charachter error:  0.2611
